In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [55]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-05 02:00:00+00:00


In [56]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [57]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-04 20:04:04,078 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 20:04:04,082 INFO: Initializing external client
2025-03-04 20:04:04,082 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 20:04:04,621 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641


In [58]:
# import sys
# import subprocess

# # Upgrade related dependencies
# subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "hsfs", "hopsworks", "protobuf"])

In [59]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.49s) 


In [60]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,230,146.0,2025-03-05 01:00:00+00:00
1,42,10.0,2025-03-05 01:00:00+00:00
2,99,0.0,2025-03-05 01:00:00+00:00
3,235,0.0,2025-03-05 01:00:00+00:00
4,63,0.0,2025-03-05 01:00:00+00:00
...,...,...,...
497,122,0.0,2025-03-05 02:00:00+00:00
498,55,0.0,2025-03-05 02:00:00+00:00
499,263,82.0,2025-03-05 02:00:00+00:00
500,89,1.0,2025-03-05 02:00:00+00:00


In [61]:
#a.info()

In [62]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-04 20:04:11,519 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 20:04:11,521 INFO: Initializing external client
2025-03-04 20:04:11,521 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 20:04:12,060 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641
Current UTC time: 2025-03-05 01:04:11.519195+00:00
Next hour: 2025-03-05 02:00:00+00:00
Querying for date range: 2025-03-05 to 2025-03-06
Filtering for hour: 2025-03-05 02:00
Found 251 records


In [63]:
now = datetime.now(timezone.utc)

In [64]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
251,165,0.0,2025-03-05 02:00:00+00:00
252,137,51.0,2025-03-05 02:00:00+00:00
253,236,103.0,2025-03-05 02:00:00+00:00
254,108,0.0,2025-03-05 02:00:00+00:00
255,207,0.0,2025-03-05 02:00:00+00:00
...,...,...,...
497,122,0.0,2025-03-05 02:00:00+00:00
498,55,0.0,2025-03-05 02:00:00+00:00
499,263,82.0,2025-03-05 02:00:00+00:00
500,89,1.0,2025-03-05 02:00:00+00:00


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  502 non-null    int32                  
 1   predicted_demand    502 non-null    float64                
 2   pickup_hour         502 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 9.9 KB


In [66]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [67]:
dt = current_date.ceil('h')


In [68]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-04 20:04:12,993 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 20:04:12,999 INFO: Initializing external client
2025-03-04 20:04:13,000 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 20:04:13,462 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.38s) 


In [69]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.41s) 


,pickup_location_id,predicted_demand,pickup_hour


In [70]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,230,146.0,2025-03-05 01:00:00+00:00
1,42,10.0,2025-03-05 01:00:00+00:00
2,99,0.0,2025-03-05 01:00:00+00:00
3,235,0.0,2025-03-05 01:00:00+00:00
4,63,0.0,2025-03-05 01:00:00+00:00
...,...,...,...
497,122,0.0,2025-03-05 02:00:00+00:00
498,55,0.0,2025-03-05 02:00:00+00:00
499,263,82.0,2025-03-05 02:00:00+00:00
500,89,1.0,2025-03-05 02:00:00+00:00


In [71]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [72]:
predictions = fetch_next_hour_predictions()

2025-03-04 20:04:19,256 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 20:04:19,258 INFO: Initializing external client
2025-03-04 20:04:19,259 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 20:04:19,797 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641
Current UTC time: 2025-03-05 01:04:19.256120+00:00
Next hour: 2025-03-05 02:00:00+00:00
Found 251 records


In [73]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-04 20:04:20,589 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 20:04:20,592 INFO: Initializing external client
2025-03-04 20:04:20,592 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 20:04:21,061 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641


In [74]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.41s) 


In [75]:
df["pickup_hour"].max()

Timestamp('2025-03-05 02:00:00+0000', tz='Etc/UTC')

In [76]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-05 02:00:00+00:00


In [77]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
251,165,0.0,2025-03-05 02:00:00+00:00
252,137,51.0,2025-03-05 02:00:00+00:00
253,236,103.0,2025-03-05 02:00:00+00:00
254,108,0.0,2025-03-05 02:00:00+00:00
255,207,0.0,2025-03-05 02:00:00+00:00
...,...,...,...
497,122,0.0,2025-03-05 02:00:00+00:00
498,55,0.0,2025-03-05 02:00:00+00:00
499,263,82.0,2025-03-05 02:00:00+00:00
500,89,1.0,2025-03-05 02:00:00+00:00
